In [1]:
import dask
from dask_jobqueue import SGECluster
from dask.distributed import Client
import dask.dataframe as dd

def limit_numpy(nthreads=4):
     """ """
     import os
     threads = str(nthreads)
     print(f"threads {threads}")
     os.environ["NUMEXPR_NUM_THREADS"] = threads
     os.environ["OMP_NUM_THREADS"] = threads
     os.environ["OPENBLAS_NUM_THREADS"] = threads
     os.environ["MKL_NUM_THREADS"] = threads
     os.environ["VECLIB_MAXIMUM_THREADS"] = threads

limit_numpy(4)

threads 4


In [2]:
cluster = SGECluster(name="dask-worker",  walltime="02:00:00", memory='5GB', death_timeout=120, project="P_lsst", resource_spec='sps=1', cores=1, processes=1)
cluster.scale(100)
client = Client(cluster)

In [3]:
data_path = "/sps/lsst/data/desc/gcr_rootdir/xgal/skysim/skysim5000_v1.1.1_parquet"

columns = [ "ra", "dec", "redshift", "halo_mass", "Mag_true_z_lsst_z0", "is_central", "halo_id"]
df = dd.read_parquet(data_path, columns=columns, engine='pyarrow', kwargs={'dataset': {'use_legacy_dataset': False}})

In [4]:
ra_min = 30
ra_max = 65
dec_min = -40
dec_max = -30

zmax = 2

cut = df.redshift < zmax
cut &= df.Mag_true_z_lsst_z0 < z0max
cut &= df.is_central == 1
cut &= df.halo_mass > mmin
cut &= (df.ra > ra_min) & (df.ra < ra_max)
cut &= (df.dec > dec_min) & (df.dec < dec_max)

In [5]:
%%time
df2 = df[cut].compute()

CPU times: user 18.2 s, sys: 1.07 s, total: 19.3 s
Wall time: 1min 27s


In [6]:
df2

,ra,dec,redshift,halo_mass,Mag_true_z_lsst_z0,is_central,halo_id
114850,32.756352,-30.079380,0.457112,8.920843e+14,-25.031357,True,721609163338
740656,40.064568,-30.253633,0.952615,7.890940e+14,-25.047729,True,3188409166253
103739,47.005075,-30.314735,0.457850,5.582540e+14,-24.986231,True,105309168338
630993,52.734021,-30.593831,0.878769,5.298776e+14,-25.084021,True,1001709170259
111173,56.120369,-30.005988,0.463085,1.048620e+15,-25.171385,True,655209171338
...,...,...,...,...,...,...,...
6924,62.485817,-38.849633,0.183383,5.544304e+14,-23.885361,True,144309942421
129509,63.960972,-38.874504,0.475919,7.123371e+14,-25.091227,True,21709942331
83390,30.952695,-39.456496,0.420646,1.111478e+15,-25.248531,True,49010059347
199634,54.374400,-39.857394,0.557466,5.475683e+14,-25.158176,True,627510067315


In [7]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
concurrent.futures._base.CancelledError
